In [51]:
# set up the openai api key
import json
import os

config_data = json.load(open("/content/config.json"))
os.environ["OPENAI_API_KEY"] = config_data["OPENAI_API_KEY"]

In [52]:
# install the required libraries
!pip install -r requirements.txt

In [50]:
# load documents
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("/content/Othman SAMIH - Rapport-1-15.pdf")
docs = loader.load()

In [53]:
# split text
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [54]:
# store splits
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(all_splits, embeddings)

In [55]:
# LLM
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [56]:
# RetrievalQA
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever()
)

In [63]:
# run chain
question = "parle moi un peu de Capgemini Engineering"
result = qa_chain({"query": question})

In [64]:
result

{'query': 'parle moi un peu de Capgemini Engineering',
 'result': 'Capgemini Engineering est une entité du groupe Capgemini spécialisée dans l\'innovation et l\'ingénierie. Elle aide ses clients à développer les produits et services de demain en les accompagnant tout au long du cycle de vie d\'un projet, de l\'idée à l\'industrialisation. Capgemini Engineering intervient dans divers secteurs tels que l\'aérospatial, l\'automobile, la défense, l\'énergie, le ferroviaire, la finance, les sciences de la vie, et les télécoms.\n\nEn 2017, Capgemini Engineering a réalisé un chiffre d\'affaires de 2,9 milliards d\'euros et compte plus de 45 000 collaborateurs répartis dans plus de 30 pays. L\'empreinte internationale de Capgemini Engineering lui permet d\'offrir des services d\'innovation et d\'ingénierie de premier ordre, en s\'appuyant sur les meilleures pratiques mondiales.\n\nLe groupe se structure autour de trois domaines d\'activités principaux, appelés "Business Lines" :\n1. **Le conse